In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/Aryand43/multimodal-llm-project.git

Cloning into 'multimodal-llm-project'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 30 (delta 4), reused 17 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 9.08 KiB | 9.08 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [3]:
%cd multimodal-llm-project

/content/multimodal-llm-project


In [4]:
!pip install -r requirements.txt
!pip install torch transformers torchvision opencv-python
!pip install openai
!pip install gradio

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 12.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement pywin32==306 (from versions: none)
ERROR: No matching distribution found for pywin32==306
  Using cached jiter-0.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.6 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:

In [6]:
import torch
import transformers
import cv2
import gradio as gr

In [7]:
!pip install gdown

file_id = '1uty8yYcAdRYe6hjBf1ohrVR_SYk0JJJV'
video_url = f'https://drive.google.com/uc?id={file_id}'

!gdown {video_url} -O Test.mp4

import cv2

video_path = 'Test.mp4'
cap = cv2.VideoCapture(video_path)

frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

cap.release()

print(f"Total frames extracted: {len(frames)}")


Downloading...
From: https://drive.google.com/uc?id=1uty8yYcAdRYe6hjBf1ohrVR_SYk0JJJV
To: /content/multimodal-llm-project/Test.mp4
100% 790k/790k [00:00<00:00, 133MB/s]
Total frames extracted: 201


In [8]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to('cuda')

captions = []
for frame in frames:
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    inputs = processor(pil_image, return_tensors="pt").to('cuda')
    output = model.generate(**inputs)
    caption = processor.decode(output[0], skip_special_tokens=True)
    captions.append(caption)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
from IPython.display import display, Image, Markdown
import numpy as np
from PIL import Image as PILImage

for idx, (frame, caption) in enumerate(zip(frames, captions)):
    pil_image = PILImage.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if idx % 3 == 0:
          display(pil_image)
          display(Markdown(f"**Frame {idx + 1} Caption**: {caption}"))

    if idx >= 31:
        print("Displaying first 5 frames only.")
        break

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!pip install ctransformers
!pip install huggingface_hub
from ctransformers import AutoModelForCausalLM
from huggingface_hub import hf_hub_download

def refine_caption(caption):
    try:

        model_path = hf_hub_download(repo_id="TheBloke/Llama-2-7B-Chat-GGUF", filename="llama-2-7b-chat.Q4_K_M.gguf")


        llm = AutoModelForCausalLM.from_pretrained(model_path, model_type="llama")


        prompt = f"""You are an AI assistant that refines image descriptions. Refine the following description:

Description: {caption}

Refined description:"""

        refined_caption = llm(prompt, max_new_tokens=200, temperature=0.7)

        refined_caption = refined_caption.split("Refined description:")[-1].strip()

        return refined_caption
    except Exception as e:
        print(f"Error refining caption: {e}")
        return None

refined_caption = refine_caption(caption)

if refined_caption:
    print("Refined Caption:", refined_caption)
else:
    print("Failed to refine caption.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 90.1 MB/s eta 0:00:00


NameError: name 'caption' is not defined